In [1]:
!pip install kaggle


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
#fetch api : 
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
#extract compressed file : 
from zipfile import ZipFile
dataset = 'sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
    zip.extractall()
    print('Dataset extracted')

Dataset extracted


In [14]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
#stopword in english : 
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
#we have to remove that word : - those word that does not impact to dataset arestopword 
#data size is very large so we will remove this 
#loading data from csv to pandas dataframe : 
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [21]:
#check the number of r and c :
twitter_data.shape

(1599999, 6)

In [22]:
#16 lakh twitt hai : 
twitter_data.head()


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [26]:
#name the col : 
column_names = ['target','id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv',names = column_names, encoding = 'ISO-8859-1')

In [27]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [30]:
#checking any missing values : 
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [32]:
#no missing values
#now distributing target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [34]:
#0 is negative 4 is positive twitt : 
#now replace 4 with one : 
twitter_data = twitter_data.replace({'target' : {4:1}})

In [37]:
twitter_data.tail()

,target,id,date,flag,user,text
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,1,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [39]:
#stemming is process of reducing word to its root word
#eg : actor,actress,acting = act 
#this is why because we have to reduce the dataset

port_stem = PorterStemmer()

In [62]:
def stemming(content):

    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [63]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [64]:
 twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [65]:
twitter_data = twitter_data.sample(frac = 1)

In [66]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
1327006,1,2015328287,Wed Jun 03 04:21:57 PDT 2009,NO_QUERY,Lauurreen,"yes, i do look like an unattractive walrus in ...",ye look like unattract walru photo
719654,0,2260664825,Sat Jun 20 19:34:57 PDT 2009,NO_QUERY,shototsu,@iamkeb I thought I could squeeze in a hike to...,iamkeb thought could squeez hike tomorrow stud...
445277,0,2068076707,Sun Jun 07 13:27:32 PDT 2009,NO_QUERY,Simon_Parker198,@JamesPMorrison I'll be damned if i can find you,jamespmorrison damn find
1076528,1,1967455041,Fri May 29 19:36:40 PDT 2009,NO_QUERY,SkinCareGoober,"Does your skin care routine keep away redness,...",skin care routin keep away red wrinkl acn mine...
1436814,1,2061064021,Sat Jun 06 20:06:28 PDT 2009,NO_QUERY,punkiefly,my Boston Terrier fell asleep to &quot;Wish&qu...,boston terrier fell asleep quot wish quot nin ...


In [67]:
X = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [69]:
print(X,y)

['ye look like unattract walru photo'
 'iamkeb thought could squeez hike tomorrow studi stack mayb yoga mon tue'
 'jamespmorrison damn find' ...
 'fill today matee proper scare iwa thorp park tomorow x'
 'rain hard hit window offic' 'im stare clubbin outfit lolll'] [1 0 0 ... 0 0 0]


In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y ,test_size=0.22, random_state=42)

In [83]:
X_train


array(['salman khan sallu danc india danc realiti show shoot boney kapoor movi salman khan sallu trivia',
       'cancel plan manch feel bad make next time',
       'sandi radbab sorri hear cheer spirit anyway', ...,
       'dlnumber fan hey',
       'clean haha go wknd next wknd parti twice much',
       'alexparde would pay sooo much come make room look dead bunni fun fill pleas swine flu top'],
      dtype=object)

In [84]:
y_train

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [85]:
#converting the textual data into numerical data :
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test  = vectorizer.transform(X_test)

In [86]:
X_train

<1248000x453868 sparse matrix of type '<class 'numpy.float64'>'
	with 9214142 stored elements in Compressed Sparse Row format>

In [87]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression()

In [88]:
pred_lr = lr.predict(X_test)

In [89]:
lr.score(X_test,y_test)

0.7758721590909091

In [90]:
#training data accuracy : 
lr.score(X_train,y_train)

0.797738782051282

In [91]:
def output_label(n) :
    if n==0 :
        return "It is a NEGATIVE TWEET"
    elif n==1 :
        return "It is a POSITIVE TWEET"

In [100]:
def manual_testing(tweet):
    testing_tweet = {'text' : [tweet]} 
    new_def_test = pd.DataFrame(testing_tweet)
    new_def_test["stemmed_content"] = new_def_test['text'].apply(stemming)
    new_x_test = new_def_test['stemmed_content']
    
    # Ensure the text is vectorized before prediction
    new_Xv_test = vectorizer.transform(new_x_test)  # Assuming vectorizer is pre-defined and fitted
    
    pred_lr = lr.predict(new_Xv_test)
    
    return "lr Prediction: {} ".format(output_label(pred_lr[0]))

In [104]:
tweet_text = str(input())


 'yes, i do look like an unattractive walrus in my photo '


In [105]:
manual_testing(tweet_text)

'lr Prediction: It is a POSITIVE TWEET '

In [103]:
twitter_data.iloc[0,]

'yes, i do look like an unattractive walrus in my photo '